In [ ]:
TELEGRAM_TOKEN = 'YOUR_TELEGRAM_TOKEN'

In [ ]:
!git clone --filter=blob:none --no-checkout https://github.com/gauris26/Machine-Learning.git
%cd Machine-Learning
!git sparse-checkout init --cone
!git sparse-checkout set chilean-school-zone-clustering-telegram-bot/resources
!git checkout main
%cd ..

In [2]:
!pip install python-telegram-bot==13.7

In [3]:
# Import necessary libraries
import pickle
import os
import signal
import numpy as np
import pandas as pd
import logging
import sys
from telegram import Update, BotCommand,ParseMode
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [4]:
class ChileSchoolClustering:
    def __init__(self, model_path, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
        self.model_path = model_path
        self.chilean_school_kmeans = None
        self.mapping_dict = {0: 'Norte Chico', 1: 'Zona Sur', 2: 'Norte Grande', 3: 'Zona Central', 4: 'Zona Austral'}
        self.cluster_centers = None
        self.cluster_labels = None

    def load_model(self):
        with open(self.model_path, 'rb') as file:
            self.chilean_school_kmeans = pickle.load(file)
        self.cluster_centers = self.chilean_school_kmeans.cluster_centers_
        self.cluster_labels = self.chilean_school_kmeans.labels_

    def predict_cluster(self):
        input_data = pd.DataFrame([[self.latitude, self.longitude]], columns=self.chilean_school_kmeans.feature_names_in_)
        cluster_prediction = self.chilean_school_kmeans.predict(input_data)
        return cluster_prediction[0]

    def get_mapped_prediction(self, cluster_prediction):
        return self.mapping_dict[cluster_prediction]

    def process(self):
        self.load_model()
        cluster_prediction = self.predict_cluster()
        mapped_prediction = self.get_mapped_prediction(cluster_prediction)

        result = {
            'Unique Clusters': np.unique(self.cluster_labels),
            'Cluster Prediction': cluster_prediction,
            'Mapped Prediction': mapped_prediction,
            'Is Anomalous': not self.is_within_chile(self.latitude, self.longitude) #anomalies and (anomalies[0] > (len(np.unique(self.cluster_labels)) - 1))
        }
        return result

    def is_within_chile(self, latitude, longitude):
        # Define the bounding box coordinates for mainland Chile
        mainland_min_latitude = -56.0
        mainland_max_latitude = -17.5
        mainland_min_longitude = -75.6
        mainland_max_longitude = -66.4

        # Define the bounding box coordinates for Easter Island
        easter_island_min_latitude = -27.2
        easter_island_max_latitude = -27.0
        easter_island_min_longitude = -109.5
        easter_island_max_longitude = -109.2

        # Define the bounding box coordinates for the Chilean Antarctic Territory
        antarctic_min_latitude = -90.0
        antarctic_max_latitude = -60.0
        antarctic_min_longitude = -74.0
        antarctic_max_longitude = -53.0

        # Define the bounding box coordinates for Juan Fernández Islands
        juan_fernandez_min_latitude = -34.0
        juan_fernandez_max_latitude = -33.6
        juan_fernandez_min_longitude = -80.9
        juan_fernandez_max_longitude = -78.8

        # Define the bounding box coordinates for Desventuradas Islands
        desventuradas_min_latitude = -27.2
        desventuradas_max_latitude = -26.3
        desventuradas_min_longitude = -80.0
        desventuradas_max_longitude = -79.0

        # Check if the given latitude and longitude are within any of the bounding boxes
        if (mainland_min_latitude <= latitude <= mainland_max_latitude and
            mainland_min_longitude <= longitude <= mainland_max_longitude):
            return True
        elif (easter_island_min_latitude <= latitude <= easter_island_max_latitude and
              easter_island_min_longitude <= longitude <= easter_island_max_longitude):
            return True
        elif (antarctic_min_latitude <= latitude <= antarctic_max_latitude and
              antarctic_min_longitude <= longitude <= antarctic_max_longitude):
            return True
        elif (juan_fernandez_min_latitude <= latitude <= juan_fernandez_max_latitude and
              juan_fernandez_min_longitude <= longitude <= juan_fernandez_max_longitude):
            return True
        elif (desventuradas_min_latitude <= latitude <= desventuradas_max_latitude and
              desventuradas_min_longitude <= longitude <= desventuradas_max_longitude):
            return True
        else:
            return False

In [5]:
# Create the Updater and pass it your bot's token.
updater = Updater(TELEGRAM_TOKEN)
resource_folder = '/content/Machine-Learning/chilean-school-zone-clustering-telegram-bot/resources'
images_folder = f'{resource_folder}/images'

# Define command handlers
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    update.message.reply_html("<b>Servicio de Zonificación Escolar Chileno (ZECH)</b> 🇨🇱")
    caption = "Bienvenido al Servicio <b>ZECH</b>"
    image_path = f'{images_folder}/Mapa_zonas_naturales_de_Chile.jpg'
    update.message.reply_photo(
        photo=open(image_path, 'rb'), 
        caption=caption,
        parse_mode=ParseMode.HTML
        )
    update.message.reply_text('Para iniciar la clusterización, por favor suministrar una geolocalización desde la opción de adjuntar ↘️')

def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text('No es una opción válida ❤️‍🩹, favor enviar una geolocalización📍')

def stop(update: Update, context: CallbackContext) -> None:
    """Stop the bot."""
    update.message.reply_text('Deteniendo el bot...')
    os.kill(os.getpid(), signal.SIGINT)

def handle_location(update: Update, context: CallbackContext) -> None:
    """Handle the user location."""
    user_location = update.message.location
    latitude = user_location.latitude
    longitude = user_location.longitude
    response_message = f"Hemos recibido su geolocalización📍:\n<b>Latitud</b>: {latitude}\n<b>Longitud</b>: {longitude}"
    update.message.reply_html(response_message)

    update.message.reply_text("Clusterizando ⚙️ ...")
    
    model_path = f'{resource_folder}/chilean_schools_clustering.pickle'

    clustering = ChileSchoolClustering(model_path, latitude, longitude)
    is_anomalous = not clustering.is_within_chile(latitude, longitude)

    if is_anomalous:
      update.message.reply_html("Se ha detectado una <b>Ubicación Fuera de Rango</b>⚠️")
    else:
      result = clustering.process()
      cluster_prediction = result['Cluster Prediction']
      mapped_prediction = result['Mapped Prediction']

      images_chile_zones = [
        f'{images_folder}/Mapa_zonas_naturales_de_Chile_Norte_Chico.jpg',
        f'{images_folder}/Mapa_zonas_naturales_de_Chile_Zona_Sur.jpg',
        f'{images_folder}/Mapa_zonas_naturales_de_Chile_Norte_Grande.jpg',
        f'{images_folder}/Mapa_zonas_naturales_de_Chile_Zona_Central.jpg',
        f'{images_folder}/Mapa_zonas_naturales_de_Chile_Zona_Austral.jpg',
      ]

      caption = f"<b>Zona Encontrada</b> ✅: {mapped_prediction} 🇨🇱"
      image_path = images_chile_zones[cluster_prediction]
      update.message.reply_photo(
          photo=open(image_path, 'rb'), 
          caption=caption, 
          parse_mode=ParseMode.HTML
          )

def main() -> None:
    """Start the bot."""

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Register handlers
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("stop", stop))
    dispatcher.add_handler(MessageHandler(Filters.location, handle_location))
    dispatcher.add_handler(MessageHandler(Filters.all & ~Filters.command, echo))

    commands = [
        BotCommand("start", "Iniciar bot"),
        BotCommand("stop", "Detener bot"),
    ]

    #Set custom commands for the bot
    updater.bot.set_my_commands(commands)

    updater.start_polling()

    updater.idle()

# Run the main function
if __name__ == '__main__':
    main()